In [ ]:
import pandas, math, numpy, time
from bokeh.layouts import gridplot, row, column
from bokeh.io import output_file, show
from bokeh.plotting import figure

In [ ]:
SData = pandas.read_excel('../../Final Data/Sensor Data.xlsx')
MData = pandas.read_excel('../../Final Data/Meteorological Interpolated.xlsx')

FactoriesInput = pandas.read_csv('../../Final Data/Factories.csv')
SensorsInput = pandas.read_csv('../../Final Data/Sensors.csv')

factories = {}
sensors = {}

for index,row in FactoriesInput.iterrows():
    factories[row['Factory']] = [row['X'],row['Y']]
for index,row in SensorsInput.iterrows():
    sensors[row['Sensor']] = [row['X'],row['Y']]
    
chemicals = ['Methylosmolene','Chlorodinine','AGOC-3A','Appluimonia']

In [6]:
output_file('all.html')
plots = []

for sensor in range(1,10):
    p1 = figure(plot_width=250,plot_height=250,title='%s %s'%(sensor,'Methylosmolene'))#, y_range = (0,5))
    p2 = figure(plot_width=250,plot_height=250,title='%s %s'%(sensor,'Chlorodinine'))#, y_range = (0,5))
    p3 = figure(plot_width=250,plot_height=250,title='%s %s'%(sensor,'AGOC_3A'))#, y_range = (0,5))
    p4 = figure(plot_width=250,plot_height=250,title='%s %s'%(sensor,'Appluimonia'))#, y_range = (0,5))

    for chemical in chemicals:
        readings = pandas.Series(SData.loc[(SData['Monitor'] == sensor) & (SData['Chemical'] == chemical)]['Reading']).values
        timestamps = pandas.Series(SData.loc[(SData['Monitor'] == sensor) & (SData['Chemical'] == chemical)]['Timestamp']).values
        timestamps = [i for i in range(len(readings))]
        if chemical == 'Methylosmolene':
            p1.circle(timestamps,readings)
        elif chemical == 'Chlorodinine':
            p2.circle(timestamps,readings)
        elif chemical == 'AGOC-3A':
            p3.circle(timestamps,readings)
        elif chemical == 'Appluimonia':
            p4.circle(timestamps,readings)
    plots.append([p1,p2,p3,p4])
    
    
grid = gridplot([[plots[i][0] for i in range(9)],[plots[i][1] for i in range(9)],[plots[i][2] for i in range(9)],[plots[i][3] for i in range(9)]],title='Everything')
show(grid)

In [7]:
WindLinear = [float(i) for i in MData['Wind Direction Linear'].values]
WindSpline = [float(i) for i in MData['Wind Direction Spline'].values]

def getAngles(factories,sensors):
    angles = {}
    for factorie in factories:
        for sensor in sensors:
            dx = sensors[sensor][0] - factories[factorie][0]
            dy = sensors[sensor][1] - factories[factorie][1]
            angle = math.degrees(numpy.arctan2(dy,dx))
            if angle < 0:
                angle = 360 + angle            
            angles['%s_%s'%(factorie,sensor)] = angle
    return angles

angles = getAngles(factories,sensors)

def makeRanges(angles,X):
    ranges = {}
    for angle in angles:
        if angles[angle] < X:
            Xmin = angles[angle] - X + 360
        else:
            Xmin = angles[angle] - X
        Xmax = angles[angle] + X
        ranges[angle] = [Xmin,Xmax]
    return ranges
        
ranges = makeRanges(angles,5)

def convertWindDirection(MData):
    directionsNew = {}
    for index, row in MData.iterrows():
        windDirection = 270 - row['Wind Direction Linear'] 
        if windDirection < 0:
            windDirection += 180
        directionsNew[row['Timestamp']] = windDirection
    return directionsNew

windDirections = convertWindDirection(MData)

def findOverlap(windDirections,ranges):
    overlap = []
    for direction in windDirections:
        d = windDirections[direction]
        for r in ranges:
            Xmin = ranges[r][0]
            Xmax = ranges[r][1]
            if Xmin > Xmax:
                if (d >= Xmin) | (d <= Xmax):
                    overlap.append('%s_%s'%(r,direction))
            else:
                if (d >= Xmin) & (d <= Xmax):
                    overlap.append('%s_%s'%(r,direction))
    return overlap
        
overlap = findOverlap(windDirections,ranges)


def overlapPerFactoriePerSensor(overlap):
    overlapNew = {}
    for factorie in factories:
        overlapNew[factorie] = {}
        for sensor in range(1,10):
            overlapNew[factorie][sensor] = []
            for o in overlap:
                if (o[0:3] == factorie) & (o[10] == str(sensor)):                
                    overlapNew[factorie][sensor].append(o[12:])
    return overlapNew    

overlap = overlapPerFactoriePerSensor(overlap)


In [8]:
def plotGraphs(SData,overlap,chemicals):
    for factorie in overlap:
        output_file('%s.html'%factorie)
        plots = []
                
        for sensor in range(1,10):
            print('Ik begin nu aan sensor %i'%sensor)
            
            Methylosmolene = []
            MethylosmoleneTS = []
            MethylosmoleneMean = numpy.mean(pandas.Series(SData.loc[(SData['Monitor'] == sensor) & (SData['Chemical'] == 'Methylosmolene')]['Reading']).values)
            
            Chlorodinine = []
            ChlorodinineTS = []
            ChlorodinineMean = numpy.mean(pandas.Series(SData.loc[(SData['Monitor'] == sensor) & (SData['Chemical'] == 'Chlorodinine')]['Reading']).values)
            
            AGOC_3A = []
            AGOC_3ATS = []
            AGOC_3AMean = numpy.mean(pandas.Series(SData.loc[(SData['Monitor'] == sensor) & (SData['Chemical'] == 'AGOC-3A')]['Reading']).values)
            
            Appluimonia = []
            AppluimoniaTS = []
            AppluimoniaMean = numpy.mean(pandas.Series(SData.loc[(SData['Monitor'] == sensor) & (SData['Chemical'] == 'Appluimonia')]['Reading']).values)
            
            p1 = figure(plot_width=250,plot_height=250,title='%s %s'%(sensor,'Methylosmolene'))#, y_range = (0,5))
            p2 = figure(plot_width=250,plot_height=250,title='%s %s'%(sensor,'Chlorodinine'))#, y_range = (0,5))
            p3 = figure(plot_width=250,plot_height=250,title='%s %s'%(sensor,'AGOC_3A'))#, y_range = (0,5))
            p4 = figure(plot_width=250,plot_height=250,title='%s %s'%(sensor,'Appluimonia'))#, y_range = (0,5))

            timestamps = overlap[factorie][sensor]
            print('%i x overlap tussen %s en sensor %i'%(len(timestamps),factorie,sensor))
            
            
            for index, row in SData[(SData['Monitor'] == sensor) & (SData['Timestamp'].isin(timestamps)) & (SData['Reading'] < 5)].iterrows():
                if str(row['Timestamp'] in timestamps):
                    
                    if row['Chemical'] == 'Methylosmolene':
                            Methylosmolene.append(row['Reading'])
                            MethylosmoleneTS.append(row['Timestamp'])
                            
                    if row['Chemical'] == 'Chlorodinine':
                        Chlorodinine.append(row['Reading'])
                        ChlorodinineTS.append(row['Timestamp'])
                            
                    if row['Chemical'] == 'AGOC-3A':
                        AGOC_3A.append(row['Reading'])
                        AGOC_3ATS.append(row['Timestamp'])
                            
                    if row['Chemical'] == 'Appluimonia':
                        Appluimonia.append(row['Reading'])
                        AppluimoniaTS.append(row['Timestamp'])
                        
            print('Me: %i'%len(Methylosmolene))
            print('Ch: %i'%len(Chlorodinine))
            print('AG: %i'%len(AGOC_3A))
            print('Ap: %i'%len(Appluimonia))
                            
            p1.circle(MethylosmoleneTS, Methylosmolene, color='orange')
            p1.line(MethylosmoleneTS,[MethylosmoleneMean for i in range(len(MethylosmoleneTS))])
            
            p2.circle(ChlorodinineTS, Chlorodinine, color='orange')
            p2.line(ChlorodinineTS,[ChlorodinineMean for i in range(len(ChlorodinineTS))])
            
            p3.circle(AGOC_3ATS, AGOC_3A, color='orange')
            p3.line(AGOC_3ATS,[AGOC_3AMean for i in range(len(AGOC_3ATS))])
            
            p4.circle(AppluimoniaTS, Appluimonia, color='orange')
            p4.line(AppluimoniaTS,[AppluimoniaMean for i in range(len(AppluimoniaTS))])
            
            plots.append([p1,p2,p3,p4])
        
        print('En dan nu plotten')
        grid = gridplot([[plots[i][0] for i in range(9)],[plots[i][1] for i in range(9)],[plots[i][2] for i in range(9)],[plots[i][3] for i in range(9)]],title='Everything')
        show(grid)
                    
plotGraphs(SData,overlap,chemicals)
        


Ik begin nu aan sensor 1
4 x overlap tussen RFE en sensor 1
Me: 4
Ch: 4
AG: 4
Ap: 4
Ik begin nu aan sensor 2
161 x overlap tussen RFE en sensor 2
Me: 159
Ch: 160
AG: 161
Ap: 161
Ik begin nu aan sensor 3
157 x overlap tussen RFE en sensor 3
Me: 137
Ch: 152
AG: 151
Ap: 155
Ik begin nu aan sensor 4
126 x overlap tussen RFE en sensor 4
Me: 107
Ch: 119
AG: 124
Ap: 126
Ik begin nu aan sensor 5
76 x overlap tussen RFE en sensor 5
Me: 63
Ch: 70
AG: 75
Ap: 76
Ik begin nu aan sensor 6
0 x overlap tussen RFE en sensor 6
Me: 0
Ch: 0
AG: 0
Ap: 0
Ik begin nu aan sensor 7
9 x overlap tussen RFE en sensor 7
Me: 7
Ch: 9
AG: 9
Ap: 9
Ik begin nu aan sensor 8
13 x overlap tussen RFE en sensor 8
Me: 12
Ch: 13
AG: 13
Ap: 13
Ik begin nu aan sensor 9
52 x overlap tussen RFE en sensor 9
Me: 46
Ch: 51
AG: 55
Ap: 52
En dan nu plotten
Ik begin nu aan sensor 1
54 x overlap tussen KOF en sensor 1
Me: 53
Ch: 54
AG: 54
Ap: 54
Ik begin nu aan sensor 2
150 x overlap tussen KOF en sensor 2
Me: 148
Ch: 148
AG: 148
Ap: 14

In [9]:
from tabulate import tabulate

table = []

for factorie in overlap:
    for chemical in chemicals:
        numOverlaps = 0
        numCorrect = 0
        for sensor in [1,2,3,5,6,7,8,9]:
            timestamps = overlap[factorie][sensor]
            mean = numpy.mean(SData[(SData['Monitor'] == sensor) & (SData['Chemical'] == chemical) & (SData['Reading'] < 5)]['Reading'].values)
            readingsOverlapping = SData[(SData['Monitor'] == sensor) & (SData['Chemical'] == chemical) & (SData['Timestamp'].isin(timestamps))]['Reading'].values
            for o in readingsOverlapping:
                if o > mean:
                    numCorrect += 1
                numOverlaps += 1
        table.append([factorie,chemical,numOverlaps,int((numCorrect/numOverlaps)*100)])
            
print(tabulate(table,headers=['factorie','chemicals','# overlaps','% correct']))

factorie    chemicals         # overlaps    % correct
----------  --------------  ------------  -----------
RFE         Methylosmolene           439           44
RFE         Chlorodinine             472           51
RFE         AGOC-3A                  505           52
RFE         Appluimonia              472           39
KOF         Methylosmolene           560           41
KOF         Chlorodinine             603           45
KOF         AGOC-3A                  646           49
KOF         Appluimonia              603           43
RCT         Methylosmolene           527           35
RCT         Chlorodinine             531           39
RCT         AGOC-3A                  537           44
RCT         Appluimonia              531           42
ISB         Methylosmolene           704           39
ISB         Chlorodinine             718           38
ISB         AGOC-3A                  732           39
ISB         Appluimonia              718           42
